In [0]:
!pip install lightgbm

In [0]:
%pip install databricks-feature_engineering

In [0]:
 %restart_python

### here we will fetch latest model from mlflow run it on dataset created from feature store of top50 features whereas 

### for production model we will load production model using champion tag then do inference on dataset create from delta table we created of production model features

### finally we will compare the accuracy of both the models

In [0]:
import pandas as pd
import lightgbm as lgb
import pickle
import joblib
import mlflow
import mlflow.lightgbm
from databricks.feature_engineering import FeatureEngineeringClient
from databricks.feature_engineering import FeatureLookup
from sklearn.metrics import accuracy_score
import numpy as np
import requests
from sklearn.metrics import (
    roc_auc_score,
    precision_score,
    recall_score
)
from databricks.sdk.service.jobs import JobSettings
from databricks.sdk import WorkspaceClient
import mlflow
import json
# Define job settings with approval gate


In [0]:
# Create a Databricks notebook widget to accept a CSV file path as input
# This allows the notebook to be parameterized and reused
dbutils.widgets.text("training_csv", "")

In [0]:

# Read the value provided to the widget (path to the training CSV file)
# Example value: /dbfs/mnt/data/training_data.csv
input_path = dbutils.widgets.get("training_csv")

In [0]:
print(input_path)

In [0]:
#loading csv
df=pd.read_csv(input_path)

In [0]:
fe = FeatureEngineeringClient()

we are loading data for inference  for production model because features  might be different in production model than latest model

we are using table ispl_databricks.model_logs.bd_final_inference_data the table we used to save the data of production model 

##### loading features of champion model

In [0]:
# Create a Spark DataFrame containing only the required label columns (loan_id and target)
df_label = spark.createDataFrame(df[['loan_id','target']])
# Load the production inference data from the Delta table into a Spark DataFrame
spark_df_production = spark.table('ispl_databricks.model_logs.bd_final_inference_data')
# Join production inference data with label data on loan_id to enrich records with target values
df_production = spark_df_production.join(df_label, on='loan_id', how='inner')
# Convert the joined Spark DataFrame to a Pandas DataFrame for further analysis or evaluation
df_production = df_production.toPandas()

##### create input ,labels to compare accuracy

In [0]:
test_production_data = df_production.drop(['loan_id','target'],axis=1)
test_production_target = df_production[['loan_id','target']]

In [0]:
fe = FeatureEngineeringClient()

### loading features for latest model using frature store table "ispl_databricks.model_logs.bd_final_feature_stores" that we create evertime during training

In [0]:
training_set = fe.create_training_set(
    df=df_label,
    feature_lookups=[
        FeatureLookup(
            table_name="ispl_databricks.model_logs.bd_final_feature_stores",
            lookup_key="loan_id"
        )
    ],
    label="target",
   
)


In [0]:
train_df = training_set.load_df()

In [0]:
train_df = train_df.toPandas()

In [0]:
test_target = train_df[['loan_id','target']]

In [0]:
test_data = train_df.drop(['loan_id','target'], axis=1)

In [0]:
train_df.shape

### LOADING  MODELS

In [0]:
from mlflow.tracking import MlflowClient

In [0]:

client = MlflowClient()

In [0]:
model_name = 'ispl_databricks.model_logs.final_bd_model'

### Fetching latest model

In [0]:
# Search for all versions of a registered MLflow model
# filter_string ensures we only fetch versions for the given model name
model_versions = client.search_model_versions(
    filter_string=f"name = '{model_name}'",
    
    
)

In [0]:

# Initialize an empty list to store model version numbers
versions = []

# Extract version numbers from the search results
# mv.version is returned as a string, so we convert it to int
for mv in model_versions:
    versions.append(int(mv.version))

In [0]:
versions

In [0]:

# Sort model versions in descending order
# This ensures the latest version appears first
versions.sort(reverse=True)

In [0]:
# Get the latest model version as a string

latest_version = str(versions[0])

In [0]:
latest_version

### production Model

In [0]:


# Load the production (champion) model from MLflow Model Registry
# '@champion' refers to the model alias pointing to the production-ready version
production_model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}@champion"
)

In [0]:
for mv in model_versions:
    print(
        f"Model: {mv.name}",
        f"Version: {mv.version}",
        f"Stage: {mv.current_stage}",
        f"Created at: {mv.creation_timestamp}",
        sep=" | "
    )

In [0]:
# Construct the MLflow model URI for a specific version of the registered model
# Format: models:/<model_name>/<version>
model_uri_latest = f'models:/ispl_databricks.model_logs.final_bd_model/{latest_version}'

# Load the specified version of the model as a PyFunc model
# This returns a callable model object that supports .predict()
model_latest = mlflow.pyfunc.load_model(model_uri_latest)

In [0]:
# Retrieve metadata (ModelInfo) for the specified MLflow model version
# This includes signature, flavors, run_id, artifact path, etc.
model_info = mlflow.models.get_model_info(model_uri_latest)

In [0]:

# Extract the input feature names from the model signature
# The signature defines the expected schema for model inputs
# input_names() returns a list of column names in the correct order
feature_names = model_info.signature.inputs.input_names()

In [0]:
len(feature_names)

### INFERENCE 

In [0]:
# Run inference using the latest trained model
# Ensure only the required features (from model signature) are passed
latest_prediction = model_latest.predict(test_data[feature_names])

# Run inference using the current production (champion) model
production_prediction = production_model.predict(test_production_data)

In [0]:
latest_prediction 

In [0]:

# Convert predicted probabilities into binary class labels (0/1)
# Threshold = 0.5
latest_result = []
for i in latest_prediction:
    if i[0]>0.5:
        latest_result.append(1)
    else:
        latest_result.append(0)
production_result = []
for i in production_prediction:
    if i[0]>0.5:
        production_result.append(1)
    else:
        production_result.append(0)

### Model evaluation metrics

In [0]:


# Accuracy: proportion of correctly classified samples
latest_model_accuracy = accuracy_score(test_target['target'].values,np.array(latest_result))
model_production_accuracy = accuracy_score(test_production_target['target'].values,np.array(production_result))
# AUC: ability of the model to distinguish between classes
latest_model_auc = roc_auc_score(test_target['target'].values,np.array(latest_result))
model_production_auc = roc_auc_score(test_production_target['target'].values,np.array(production_result))


# Precision: how many predicted positives are actually positive

latest_model_precision = precision_score(test_target['target'].values,np.array(latest_result))
model_x_precision = precision_score(test_production_target['target'].values,np.array(production_result))

# Recall: how many actual positives were correctly predicted
latest_model_recall = recall_score(test_target['target'].values,np.array(latest_result))
model_x_recall = recall_score(test_production_target['target'].values,np.array(production_result))

In [0]:
print("latest accuracy",latest_model_accuracy)
print("model_production_accuracy",model_production_accuracy)


### compare accuracy between production model and latest model

In [0]:
json_dict = {
    "model_name": model_name,
    'new_accuracy': latest_model_accuracy,
    'new_auc': latest_model_auc,
    'latest_version':latest_version
    }

In [0]:
with open("/Workspace/Shared/ff_bd/model_approval.json", "w") as f:
    json.dump(json_dict, f)


In [0]:
if latest_model_accuracy > model_production_accuracy:
    notification = 'True'
else:
    notification = 'False'

In [0]:
dbutils.jobs.taskValues.set("trigger", notification)